In [1]:
from quchem.Ansatz_Generator_Functions import *

## Theory

The HF technique does not take into account electron correlation effects, other than the Pauli exclusion principle which is imposed by the Slater determinant. 

To correct for this, the wavefunction can be expanded as a superposition of all the determinants in the N electron Fock Space.

The total number of determinants is given by the binomial:

$$\binom{M}{N} = \frac{M!}{N!(M-N)!}$$

- $M=$ number of spin orbitals
- $N=$ number of e- 

for example in H2 in an STO-3G basis there are 4 orbitals and 2 electrons

therefore there are $\binom{4}{2} = 6$ determinants. The superposition over all determinants is: $\alpha |{1100}\rangle + \beta |{1010}\rangle + \gamma |{1001}\rangle, \delta |{0110}\rangle + \epsilon |{0101}\rangle + \zeta |{0011}\rangle$.  

The importance of this is that as all possible determinants are present, the exact wavefunction of any state in the system can be written in this form!

Including all the determinants will yield the full configuration interaction (FCI) wavefunction, if all the excitations above the HF wavefunction are considered. 

 This can be formalized with the definition of the excitation operators:

$$T = \sum_{i=1}^{N} T_{i}$$

where:

$$T_{1} = \sum_{\substack{i\in occ \\  \alpha \in virt}} t_{\alpha}^{i}a_{\alpha}^{\dagger}a_{i}$$



$$T_{2} = \sum_{\substack{i>j\in occ, \\  \alpha > \beta \in virt}} t_{\alpha \beta}^{ij}a_{\alpha}^{\dagger}a_{\beta}^{\dagger}a_{i}a_{j}$$

$$\dots$$ 

note:
- occ = occupied spin orbitals of **reference state**
- virt = unoccupied spin orbitals of **reference state**

Single excitations from the reference stateare generated by $T_{1}$, $T_{2}$ double excitations and higher order terms follow as expected. 
 
The expansion coefficients are denoted as $t_{\alpha}^{i}$ and $t_{\alpha \beta}^{ij}$ and determine the amplitude values!

The unitary coupled cluster wavefunction is defined as:


$$|\psi_{UCC}\rangle = e^{T-T^{\dagger}}|\psi_{HF}\rangle$$


where

$U_{U C C}(\vec{t})=e^{T-T^{\dagger}}=e^{\sum_{i} t_{i}\left(T_{i}-T_{i}^{\dagger}\right)}$

## UCCSD

UCCSD only takes into account **single** and **double** electron excitations:

$$U_{CCSD} =e^{\left(T_{1}-T_{1}^{\dagger}\right)+\left(T_{2}-T_{2}^{\dagger}\right)}$$

$$T_{1} = \sum_{\substack{i\in occ \\  \alpha \in virt}} t_{\alpha}^{i}a_{\alpha}^{\dagger}a_{i}$$



$$T_{2} = \sum_{\substack{i>j\in occ, \\  \alpha > \beta \in virt}} t_{\alpha \beta}^{ij}a_{\alpha}^{\dagger}a_{\beta}^{\dagger}a_{i}a_{j}$$

Leads to more TRACTABLE calculation


In [2]:
num_electrons = 2
num_orbitals = 4
ansatz = Ansatz(num_electrons, num_orbitals)

ia_terms, ijab_terms, ia_theta, ijab_theta = ansatz.Get_ia_and_ijab_terms()

print(ia_terms)
print('')
print(ijab_terms)

[-1.0 [0^ 2] +
1.0 [2^ 0], -1.0 [1^ 3] +
1.0 [3^ 1]]

[-1.0 [0^ 1^ 2 3] +
1.0 [3^ 2^ 1 0]]


## Suzuki-Trotter Decomposition

Often we are given an operator as a sum of operstors (Just the excitation operators of the UCC operator $T = \sum_{i=1}^{N} T_{i}$)... Here we are considering a Hamiltonian:

$$H = H_{1} + H_{2}$$

from the **second postulate of QM** we want to perform the time evolution of the quantum system (using the Hamiltonian). When $H$ is constant in time the evolution operator $U(t)$ has a very simple form:

$$U(t)=e^{-i \frac{H}{\hbar}t}$$

**IMPORTANT note about exponentiated operators**

Exponentiated operators DO NOT SHARE ALL PROPERTIES FOR EXPONENTIATED NUMBERS

e.g.
- for numbers we have: $e^{a+b} = e^{a}e^{b}$
- for operators in general $e^{\hat{A}+\hat{B}} \neq e^{\hat{A}}e^{\hat{B}}$

Only obeys this relationship if and only if ($\iff$) the operators commute:
$$e^{\hat{A}+\hat{B}} = e^{\hat{A}}e^{\hat{B}} \iff [\hat{A}, \hat{B}]=0$$



Therefore if  $H_{1}$ and $H_{2}$ do not commute, we could simulate them individually and add the results... BUT this will NOT calculate the full $H$.

In such cases one can use the Suzuki-Trotter decomposition, where for any operators $\hat{A}$ and $\hat{B}$:

$$e^{\hat{A}+\hat{B}}=\lim _{n \rightarrow \infty}\left(e^{\hat{A} / n} e^{\hat{B} / n}\right)^{n}=\lim _{n \rightarrow \infty}\left(e^{\hat{B} / n} e^{\hat{A} / n}\right)^{n}$$

therefore for QUANTUM evolution operators:

$$U(t)=e^{\frac{-i t}{\hbar}\left(H_{1}+H_{2}\right)}=\lim _{n \rightarrow \infty}\left(U_{1}(t / n) U_{2}(t / n)\right)^{n}$$

This can be interpretted as the full evolution of $H_{1}$ and $H_{2}$ for successive slices and as the slices become infinitessimal the approximation becomes exact

## Single trotter step

$$U_{U C C}(\vec{t}) \approx U_{U C C}^{(T r o t)}=\left(\prod_{j} \exp{ \big( \frac{t_{j}}{\rho}\left(T_{j}-T_{j}^{\dagger}\right)\big)}\right)^{\rho}$$

- $\rho =$ trotter step

for single step ($\rho =1 $)

$$U_{U C C}^{(T r o t)}=\prod_{j} \exp{ \big( t_{j}\left(T_{j}-T_{j}^{\dagger}\right)\big)}$$

therefore for the SINGLE - DOUBLE version:

$$U_{U C C SD}^{(T r o t)}=e^{t_{1}\left(T_{1}-T_{1}^{\dagger}\right)} \times e^{t_{2}\left(T_{2}-T_{2}^{\dagger}\right)}$$

Therefore using H2 as an example:

$$U_{UCCSD} = e^{t_{02}(a_{2}^{\dagger}a_{0} - a_{0}^{\dagger}a_{2}) + t_{13}(a_{3}^{\dagger}a_{1} - a_{1}^{\dagger}a_{3}) + t_{0123}(a_{3}^{\dagger}a_{2}^{\dagger}a_{1}a_{0} - a_{0}^{\dagger}a_{1}^{\dagger}a_{2}a_{3})} = exp{\big( t_{02}(a_{2}^{\dagger}a_{0} - a_{0}^{\dagger}a_{2}) + t_{13}(a_{3}^{\dagger}a_{1} - a_{1}^{\dagger}a_{3}) + t_{0123}(a_{3}^{\dagger}a_{2}^{\dagger}a_{1}a_{0} - a_{0}^{\dagger}a_{1}^{\dagger}a_{2}a_{3}) \big)}$$


$$U_{UCCSD}^{Trot} = e^{t_{02}(a_{2}^{\dagger}a_{0} - a_{0}^{\dagger}a_{2})} \times e^{ t_{13}(a_{3}^{\dagger}a_{1} - a_{1}^{\dagger}a_{3})} \times e^{t_{0123}(a_{3}^{\dagger}a_{2}^{\dagger}a_{1}a_{0} - a_{0}^{\dagger}a_{1}^{\dagger}a_{2}a_{3})}$$

In [3]:
num_electrons = 2
num_orbitals = 4
ansatz = Ansatz(num_electrons, num_orbitals)
ia_terms, ijab_terms, ia_theta, ijab_theta = ansatz.Get_ia_and_ijab_terms()

SecQuantCCsingle_Trot_list_ia_JW, SecQuantCCsingle_Trot_list_ijab_JW = ansatz.UCCSD_single_trotter_step(ia_terms, 
                                                                                                        ijab_terms,
                                                                                                        transformation='JW')

for index, ia_term in enumerate(ia_terms):
    print('ia_term: ', ia_term)
    print('becomes:', SecQuantCCsingle_Trot_list_ia_JW[index])
    print('')
    
print('')
print('####')
print('')

for index, ijab_term in enumerate(ijab_terms):
    print('ijab_term: ', ijab_term)
    print('becomes:', SecQuantCCsingle_Trot_list_ijab_JW[index])
    print('')


ia_term:  -1.0 [0^ 2] +
1.0 [2^ 0]
becomes: -0.5j [X0 Z1 Y2] +
0.5j [Y0 Z1 X2]

ia_term:  -1.0 [1^ 3] +
1.0 [3^ 1]
becomes: -0.5j [X1 Z2 Y3] +
0.5j [Y1 Z2 X3]


####

ijab_term:  -1.0 [0^ 1^ 2 3] +
1.0 [3^ 2^ 1 0]
becomes: 0.125j [X0 X1 X2 Y3] +
0.125j [X0 X1 Y2 X3] +
-0.125j [X0 Y1 X2 X3] +
0.125j [X0 Y1 Y2 Y3] +
-0.125j [Y0 X1 X2 X3] +
0.125j [Y0 X1 Y2 Y3] +
-0.125j [Y0 Y1 X2 Y3] +
-0.125j [Y0 Y1 Y2 X3]



**note** each Paulistring in each subterm commutes and so we can use the standard exponentiated matrix simplification rules

Only obeys this relationship if and only if ($\iff$) the operators commute:
$$e^{\hat{A}+\hat{B}} = e^{\hat{A}}e^{\hat{B}} \iff [\hat{A}, \hat{B}]=0$$

Overall using a single trotter step we get:

$$U_{UCC}^{Trot} =  \\ e^{-i\frac{\theta_{20}}{2}(X_{0} Z_{1} Y_{2})} \times e^{i\frac{\theta_{20}}{2}(Y_{0} Z_{1} X_{2})} \\ \times e^{-i\frac{\theta_{31}}{2}(X_{1} Z_{2} Y_{3})} \times e^{i\frac{\theta_{31}}{2}(Y_{1} Z_{2} X_{3})}    \times   \\
    e^{\frac{\theta_{3210}}{8}(X_{0} X_{1} X_{2} Y_{3})}     \times
    e^{i\frac{\theta_{3210}}{8}(X_{0} X_{1} Y_{2} X_{3})}\times 
    e^{-i\frac{\theta_{3210}}{8}(X_{0} Y_{1} X_{2} X_{3})}\times
    e^{i\frac{\theta_{3210}}{8}(X_{0} Y_{1} Y_{2} Y_{3})}\times
    e^{-i\frac{\theta_{3210}}{8}(Y_{0} X_{1} X_{2} X_{3})}\times
    e^{i\frac{\theta_{3210}}{8}(Y_{0} X_{1} Y_{2} Y_{3})}\times
    e^{-i\frac{\theta_{3210}}{8}(Y_{0} Y_{1} X_{2} Y_{3})}\times
    e^{-i\frac{\theta_{3210}}{8}(Y_{0} Y_{1} Y_{2} X_{3})}$$
    

# Get Hartree-Fock states

#### Jordan-Wigner Transformation

In [4]:
from quchem.Ansatz_Generator_Functions import *
n_electrons=2
n_qubits=4

ansatz_obj = Ansatz(n_electrons, n_qubits)

print('full state = ',ansatz_obj.Get_JW_HF_state())
print('')
print('state in occ number basis = ',ansatz_obj.Get_BK_HF_state_in_OCC_basis())

full state =  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]

state in occ number basis =  [1. 0. 0. 0.]


#### Bravyi-Kitaev Transformation

In [5]:
print('full state = ', ansatz_obj.Get_BK_HF_state())
print('state in occ number basis = ',ansatz_obj.Get_BK_HF_state_in_OCC_basis())

full state =  [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
state in occ number basis =  [1. 0. 0. 0.]


## Get initial theta values using coupled cluster calculation

In [8]:
from quchem.Hamiltonian_Generator_Functions import *
### CLASSICAL QUANTUM CALCULATION
Molecule = 'H2'#'LiH' #'H2
geometry = None #[('Li', (0., 0., 0.)), ('H', (0., 0., 1.45))] # [('H', (0., 0., 0.)), ('H', (0., 0., 0.74))]
basis = 'sto-3g'

### Get Hamiltonian
Hamilt = Hamiltonian(Molecule,
                     run_scf=1, run_mp2=1, run_cisd=1, run_ccsd=1, run_fci=1,
                     basis=basis,
                     multiplicity=1,
                     geometry=geometry)  # normally None!

Hamilt.Get_Molecular_Hamiltonian(Get_H_matrix=False)
QubitHam = Hamilt.Get_Qubit_Hamiltonian(transformation='JW')
Ham_matrix_JW = Hamilt.Get_sparse_Qubit_Hamiltonian_matrix(QubitHam)



### Get classical CC amplitudes
Hamilt.Get_CCSD_Amplitudes()



ansatz_obj = Ansatz(Hamilt.molecule.n_electrons, Hamilt.molecule.n_qubits)

Sec_Quant_CC_ia_ops, Sec_Quant_CC_ijab_ops, theta_parameters_ia, theta_parameters_ijab = ansatz_obj.Get_ia_and_ijab_terms(single_cc_amplitudes=Hamilt.molecule.single_cc_amplitudes,
                                                                                                                        double_cc_amplitudes=Hamilt.molecule.double_cc_amplitudes,
                                                                                                                        singles_hamiltonian=Hamilt.singles_hamiltonian,
                                                                                                                    doubles_hamiltonian=Hamilt.doubles_hamiltonian,
                                                                                                                        tol_filter_small_terms = None)
theta_parameters_ijab

[-0.08903292495]

## Use NOON to remove terms for UCCSD operators

In [31]:
from quchem.Hamiltonian_Generator_Functions import *
from quchem.Graph import *
### HAMILTONIAN start
Molecule = 'LiH'
geometry = None # [('H', (0., 0., 0.)), ('H', (0., 0., 0.74))]
basis = 'sto-3g'


### Get Hamiltonian
Hamilt = Hamiltonian(Molecule,
                     run_scf=1, run_mp2=1, run_cisd=1, run_ccsd=1, run_fci=1,
                     basis=basis,
                     multiplicity=1,
                     geometry=geometry)  # normally None!
QubitHamiltonian = Hamilt.Get_Qubit_Hamiltonian(threshold=None, transformation='BK')
### HAMILTONIAN end

## Get CC ampltidues
Hamilt.Get_CCSD_Amplitudes()

##
NOON_spins_combined, NMO_basis = Hamilt.Get_NOON()
##
Hamilt.Get_CCSD_Amplitudes()
NOON_spins_combined

array([1.99991509e+00, 1.96744892e+00, 2.71958861e-02, 7.94453898e-05,
       2.68032899e-03, 2.68032899e-03])

In [34]:
ansatz_obj = Ansatz(Hamilt.molecule.n_electrons, Hamilt.molecule.n_qubits)
reduced_CC_ops_ia , reduced_CC_ops_ijab, theta_ia, theta_ijab = ansatz_obj.Remove_NOON_terms(
                            NOON=NOON_spins_combined,
                             occ_threshold=1.999,
                             unocc_threshold=1e-4,
                             indices_to_remove_list_manual=None,
                            single_cc_amplitudes=Hamilt.molecule.single_cc_amplitudes,
                            double_cc_amplitudes=Hamilt.molecule.double_cc_amplitudes,
                             singles_hamiltonian=None, 
                             doubles_hamiltonian=None, 
                             tol_filter_small_terms=None)

ia_terms, ijab_terms, ia_theta, ijab_theta = ansatz_obj.Get_ia_and_ijab_terms()
print('REDUCTION')
print('ia_terms', len(ia_terms), 'TO', len(reduced_CC_ops_ia))
print('ijab_terms', len(ijab_terms), 'TO', len(reduced_CC_ops_ijab))

REDUCTION
ia_terms 16 TO 6
ijab_terms 42 TO 6
